In [1]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import tensorflow as tf
import glob
from scipy.signal import butter, lfilter, find_peaks

c:\Users\pc\miniconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\pc\miniconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\pc\miniconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### cv2 as a ROI detector

In [15]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)
while True:
    ret, img = cap.read()
   
    if not ret:
        break
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    for (x, y, w, h) in faces:
        forehead_y = int(y + 0.25 * h)
        forehead_h = int(0.2 * h)
        cv2.rectangle(img, (x, forehead_y-50), (x + w, forehead_y + forehead_h-50), (0, 255, 0), 2)

    cv2.imshow('image', img)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

## Our model as a ROI detector

In [17]:
forehead = load_model('forehead.h5')   # Loading Our already Trained Model.

In [3]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _ , frame = cap.read()
    if not(_):
        print("not assesed")
        break
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb,(120, 120))
    
    yhat = forehead.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]
    
    if yhat[0] > 0.8: 
        cv2.rectangle(frame, 
                      tuple(np.multiply(sample_coords[:2], [640,480]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [640,480]).astype(int)), 
                            (255,0,0), 2)
    
    cv2.imshow('EyeTrack', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 156ms/step


# Prediction

In [18]:
def view_prediction(path):
    frame = cv2.imread(path)
    frame = cv2.resize(frame,(640,480))
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb,(120, 120))

    yhat = forehead.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]

    if yhat[0] >= 0.95: 
        cv2.rectangle(frame, 
                        tuple(np.multiply(sample_coords[:2], [640,480]).astype(int)),
                        tuple(np.multiply(sample_coords[2:], [640,480]).astype(int)), 
                            (255,0,0), 2)
    cv2.imshow('EyeTrack', frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [19]:
view_prediction('aug_data\\train\\images\\imagesp268.3.jpg')  # Give path to the image for which you want to see the predcition

1/1 [==============================] - 1s 1s/step


## IOU Calculation

In [6]:
def calculate_iou(box1, box2):
    x_left = max(box1[0], box2[0])
    y_top =  max(box1[1], box2[1])
    x_right = min(box1[2], box2[2])
    y_bottom = min(box1[3], box2[3])

    if x_right < x_left or y_bottom < y_top:
        return 0.0  

    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    iou = intersection_area / float(box1_area + box2_area - intersection_area)

    return iou

In [8]:
IMages = []
for filename in glob.glob('aug_data\\test\images\\*.jpg'):
    IMages.append(cv2.imread(filename))

In [13]:
real_box = []
import json
for filename in glob.glob('aug_data\\test\labels\\*.json'):
    with open(filename, 'r') as f:
        label = json.load(f)
    real_box.append(label['bbox'])

In [16]:
label['bbox']

[0.39332170050761417,
 0.5761421319796953,
 0.49325824873096447,
 0.6120981387478849]

In [15]:
error = 0
for j , i in enumerate(IMages):
    rgb = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb,(120, 120))
    yhat = forehead.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]
    # error += calculate_iou(real_box[j],np.multiply(sample_coords,[640,480,640,480]))
print("The IOU of this model is",error)

1/1 [==============================] - 0s 85ms/step
The IOU of this model is 0.68


# cv2 as a motion handling tool

In [20]:
cap = cv2.VideoCapture(0)
prev_frame = None
prev_pts = None
max_pts = 200
stability_threshold = 0.8
max_unstable_frames = 5
unstable_frames = 0

ro = []
rs = []

while True:
    ret, frame = cap.read()

    if ret:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_frame is not None:
            pts, status, error = cv2.calcOpticalFlowPyrLK(prev_frame, gray, prev_pts, None)
            good_pts = pts[status == 1]
            good_prev_pts = prev_pts[status == 1]

            if len(good_pts) < max_pts * stability_threshold:
                unstable_frames += 1
                if unstable_frames > max_unstable_frames:
                    prev_frame = None
                    prev_pts = None
                    unstable_frames = 0
            else:
                unstable_frames = 0
                M, _ = cv2.estimateAffinePartial2D(good_prev_pts, good_pts)
                frame[100:120,320:340,:] = np.full((20,20,3),255)
                rs.append(np.mean(frame[100:120,320:340,2]))
                stabilized = cv2.warpAffine(frame, M, (frame.shape[1], frame.shape[0]))
                cv2.imshow('Stabilized', stabilized)
        
        if prev_frame is None:
            pts = cv2.goodFeaturesToTrack(gray, max_pts, 0.01, 10)

            if pts is not None:
                prev_pts = pts.reshape(-1, 1, 2)
            prev_frame = gray


        frame[70:100,300:330,:] = np.full((30,30,3),0)
        ro.append(np.mean(frame[100:130,330:360,2]))
        cv2.imshow('Original', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()


# Our model as a motion handling tool.

In [21]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _ , frame = cap.read()
    if not(_):
        print("not assesed")
        break
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb,(120, 120))
    
    yhat = forehead.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]
    
    
    x1 = sample_coords[0] * 640 
    y1 = sample_coords[1] * 480
    x2 = sample_coords[2] * 640
    y2 = sample_coords[3] * 480
    
    x = int((x1 + x2) / 2)
    y = int((y1 + y2) / 2)

    frame[y-15:y+15,x-15:x+15,2] = np.zeros((30,30))

    cv2.rectangle(frame, 
                      tuple(np.multiply(sample_coords[:2], [640,480]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [640,480]).astype(int)), 
                            (255,0,0), 2)
    
    cv2.imshow('EyeTrack', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 91ms/step
